In [5]:
from numpy.polynomial import Polynomial
from numpy.polynomial.polynomial import polydiv
from phe import paillier
from numpy import prod
import secrets

In [106]:
class Player:
    """
    Class of honest-but-curious adversaries.

    Parameters
    ----------
    S : set_like
    sk : secret key to a homomorpic cryptosystem
    """
    def __init__(self, S, sk):
        self.S = S
        self.k = len(S)
        self.sk = sk
        self.f_list = []
        self.N = sk.public_key.n
    
    def polynomial(self):
        # cria o polinómio f a partir do conjunto S 
        self.f = Polynomial.fromroots(self.S)
    
    def encrypt(self, pk):
        # encipta o polinómio f com pk
        self.ef = self.f #Polynomial([pk.encrypt(x) for x in self.f.coef])                                 HERE
    
    def append_f(self, encrypted_polynomial):
        # adiciona polinomio encriptado de outro jogador a f_list
        self.f_list.append(encrypted_polynomial)

    def receive_l(self, encrypted_polynomial):
        # recebe lambda_i-1
        self.lam_im1 = encrypted_polynomial
    
    def receive_p(self, encrypted_polynomial):
        #recebe p = lambda_n
        self.ep = encrypted_polynomial

    def phi(self):
        # calcula phi_i
        c = len(self.f_list)
        r = []
        for _ in range(c):
            r.append(Polynomial([secrets.randbelow(self.N) for _ in range(self.k + 1)])) 
        self.phi_i = sum([prod([self.f_list[j],r[j]]) for j in range(c)])
        #for x in self.phi_i.coef:                                                                          HERE
        #    x.obfuscate() #re-randomização dos coeficientes

    def lam(self):
        # calcula lambda_i
        self.lam_i = self.lam_im1 + self.phi_i
        #for x in self.lam_i.coef:                                                                          HERE
        #    x.obfuscate() #re-randomização dos coeficientes
    
    def decrypt(self):
        self.p = self.ep #Polynomial([self.sk.decrypt(x) for x in self.ep.coef])                            HERE

    def multiset(self):
        # calcula o multiset de intersecção na forma [(a,b),...,(a,b)], a \in S_i
        # onde o elemento a aparece b vezes no multiset
        res = []
        for elem in self.S:
            degree = 0
            poly = self.p
            quo, rem = polydiv(poly.coef, (1, -elem))
            print(elem, quo,rem)
            while all(rem == 0):
                degree += 1
                poly = quo
                quo, rem = polydiv(poly, (1, -elem))
                print(quo,rem)
            if degree > 0:
                res.append((elem, degree))
        self.intersection = res

    

Soma de polinómios encriptados: `a.ef + b.ef`

Produto de polinímio não encriptado com polinómio encriptado: `a.f * b.ef`

Derivada de polinómio encriptado: `a.ef.deriv(d)`

Avaliação de um polinómio encriptado num ponto não encriptado: `a.ef(x)`


Função da secção 5.1

In [107]:
def psi(players, c, pk):
    """
    Perform a private Set-Intersection secure against a coalition of honest-but-curious
    adversaries.

    Parameters
    ----------
    players : array_like construct of class Player elements
    c : dishonestly colluding constant
    pk: public key to a homomorpic cryptosystem
    """
    n = len(players)
    assert n >= 2, "Not enough players"
    assert c < n, "Dishonestly colluding too large"
    assert all([player.k == players[0].k for player in players]),\
        "Differently sized private sets"
    #1
    for player in players:
        #a
        player.polynomial()       
        player.encrypt(pk)
    
    #b
    for i in range(n):
        for j in range(1, c+1):
            players[i].append_f(players[(i-j) % n].ef)
        
        # Qual é o domínio?? #consoante a resposta é preciso alterar a função phi
        # DEFAULT_KEYSIZE = 3072
        # BASE = 16
        #c, d
        players[i].phi()

    #2
    players[0].lam_i = players[0].phi_i
    players[1].receive_l(players[0].lam_i)

    #3
    for i in range(1, n):
        #a é realizado implicitamente
        #b
        players[i].lam()

        #c
        players[(i+1)%len(players)].receive_l(players[i].lam_i)
    #4
    for player in players:
        player.receive_p(players[0].lam_i)

    #5
    for player in players:
        player.decrypt()
        pass
    #6
    for player in players:
        player.multiset()

In [99]:
# Generate keys
pk, sk = paillier.generate_paillier_keypair(None, 32)
pk.max_int = pk.n

In [100]:
# Create players
a = Player([1,2,3,4,5],sk)
b = Player([1,1,1,1,1],sk)

In [111]:
pk, sk = paillier.generate_paillier_keypair(None, 32)
pk.max_int = pk.n

a = Player([1,2,3,4,5],sk)
b = Player([3,3,3,3,3],sk)

# Calculate private set
psi([a,b],1,pk)

1 [-1.82612659e+11  4.68586096e+11 -3.51197546e+11  1.01620058e+11
 -1.09904961e+11 -2.70202551e+10  1.15863552e+11 -6.42762499e+10
  1.41607112e+10 -1.13288049e+09] [-3.19608226e+11]
2 [-1.67728349e+11  3.15742058e+11 -1.88299527e+11  7.62185517e+10
 -5.90879163e+10 -3.52911263e+10  7.23015548e+10 -3.55366927e+10
  7.36357571e+09 -5.66440243e+08] [-3.34492537e+11]
3 [-1.40604701e+11  2.29384653e+11 -1.31629682e+11  5.79285584e+10
 -3.77393445e+10 -3.03333272e+10  5.18838260e+10 -2.44883243e+10
  4.97198828e+09 -3.77626829e+08] [-3.61616185e+11]
4 [-1.17918187e+11  1.79526006e+11 -1.01679617e+11  4.60991384e+10
 -2.71284659e+10 -2.56291575e+10  4.03671776e+10 -1.86710921e+10
  3.75259289e+09 -2.83220122e+08] [-3.84302699e+11]
5 [-1.00766061e+11  1.47368450e+11 -8.29413943e+10  3.81106334e+10
 -2.09718528e+10 -2.19745579e+10  3.30110181e+10 -1.50847116e+10
  3.01340311e+09 -2.26576097e+08] [-4.01454825e+11]
3 [-1.40604701e+11  2.29384653e+11 -1.31629682e+11  5.79285584e+10
 -3.77393445e

In [110]:
#find private set
for x in a.intersection:
    print(x)


In [ ]:
#get obtained provate sets

In [ ]:
#compare make sure the sets are the same

In [14]:
# Teste das operações definidas em 4.2
pk, sk = paillier.generate_paillier_keypair()
a = Player([1,2,3],sk)
b = Player([2,3,4],sk)
a.polynomial()
b.polynomial()
a.encrypt(pk)
b.encrypt(pk)

d = 2
x = 0
print(f"Soma:\n{a.ef + b.ef}")
print(f"Multiplicação:\n{a.f * b.ef}")
print(f"Derivada:\n{a.ef.deriv(d)}")
print(f"Avaliação:\n{a.ef(x)}")


Soma:
-30.0 + 37.0 x - 15.0 x**2 + 2.0 x**3
Multiplicação:
144.0 - 420.0 x + 484.0 x**2 - 285.0 x**3 + 91.0 x**4 - 15.0 x**5 +
1.0 x**6
Derivada:
-12.0 + 6.0 x
Avaliação:
-6.0
